## [reptile](https://blog.openai.com/reptile/)
we initially fit a model on ten randomly generated sine curves, each with a different phase, amplitutde, and bias. for each sine curve, 500 data points is randomly selected for training data.

we will then fit the model on a new sine curve. the challenge is learning this new sine curve with only 25 datapoints. (in other words, the model will learn a new sine curve with 95% less data.)

In [ ]:
import tensorflow as tf
import numpy as np

def generate_points(k):
    """sample k points from a randomly initializes sine curve.
    """
    phase = np.random.uniform(low=-10, high=10)
    amplitude = np.random.uniform(low=-50, high=50)
    bias = np.random.uniform(low=-10, high=10)
    frequency = np.random.uniform(low=-2, high=2)
    n_points = 5000  # total num of points to generate in sine wave; only a subset will be used as training data
    
    if (k > n_points):
        raise ValueError('oversampling')

    X = np.linspace(0, 2, n_points)
    y = amplitude*np.sin(X*frequency*2*np.pi+phase*np.pi/180.0)+bias
    
    keys = sorted(np.random.choice(np.arange(n_points), size=k))
    return (X[keys], y[keys], X, y)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

n_samples = 5
n_points_per_samples = 20  # for each sine curve, grab 50 <x,y> for training

for _ in range(n_samples):
    X_train_subset, y_train_subset, X_train, y_train = generate_points(n_points_per_samples)
    plt.plot(X_train_subset, y_train_subset, marker='o')
    plt.title("random sine curves for training")

In [ ]:
tf.reset_default_graph()

n_hidden = 128
n_classes = 1
n_features = 1

X_ = tf.placeholder(tf.float32, shape=[None, n_features])
y_ = tf.placeholder(tf.float32, shape=[None, n_classes])

w1 = tf.Variable(tf.random_uniform([n_features, n_hidden]))
b1 = tf.Variable(tf.random_uniform([n_hidden]))

w2 = tf.Variable(tf.random_uniform([n_hidden, n_classes]))
b2 = tf.Variable(tf.random_uniform([n_classes]))

z1 = tf.matmul(X_, w1) + b1
fc1 = tf.nn.sigmoid(z1)
z2 = tf.matmul(fc1, w2) + b2


loss = tf.reduce_mean(tf.nn.l2_loss(z2 - y_))
op = tf.train.AdamOptimizer(0.01).minimize(loss)

In [ ]:
"""
we have 500 data points for each sine curve.
"""

n_samples = 10  # number of sine curves
n_points_per_samples = 500  # how datapoints per sample?
n_epochs = 2000

ls_w1 = []
ls_b1 = []
ls_w2 = []
ls_b2 = []


with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    for draw in range(n_samples):
        
        # generate datapoints from data
        X_train_subset, y_train_subset, _, _ = generate_points(n_points_per_samples)
    
        # parameter update
        for i in range(n_epochs):
            _ = sess.run(op, feed_dict={X_:X_train_subset.reshape(n_points_per_samples,1), 
                                        y_:y_train_subset.reshape(n_points_per_samples,1)})
            
        # what is the final loss per sample?
        print("sample {}: loss {}".format(draw, sess.run(loss, feed_dict={
            X_:X_train_subset.reshape(n_points_per_samples,1), 
            y_:y_train_subset.reshape(n_points_per_samples,1)})))

        # collect weights for averaging
        ls_w1.append(sess.run(w1))
        ls_b1.append(sess.run(b1))
        ls_w2.append(sess.run(w2))
        ls_b2.append(sess.run(b2))
        
    # generate predictions for visual inspection
    y_pred = sess.run(z2, feed_dict={X_:X_train_subset.reshape(n_points_per_samples,1)})

In [ ]:
plt.plot(X_train_subset, y_train_subset, c='g')  # actual
plt.plot(X_train_subset, y_pred, c='b')  # pred
plt.title("actual sine curve versus predicted")

In [ ]:
# calculate the average of weights for each sample

w1_new = np.mean(ls_w1, axis=0)
b1_new = np.mean(ls_b1, axis=0)
w2_new = np.mean(ls_w2, axis=0)
b2_new = np.mean(ls_b2, axis=0)

In [ ]:
# create a new model for few-shot learning on new sine curve

tf.reset_default_graph()

n_hidden = 128
n_classes = 1
n_features = 1

X_ = tf.placeholder(tf.float32, shape=[None, n_features])
y_ = tf.placeholder(tf.float32, shape=[None, n_classes])

w1 = tf.Variable(tf.random_uniform([n_features, n_hidden]))
b1 = tf.Variable(tf.random_uniform([n_hidden]))
w2 = tf.Variable(tf.random_uniform([n_hidden, n_classes]))
b2 = tf.Variable(tf.random_uniform([n_classes]))

z1 = tf.matmul(X_, w1) + b1
fc1 = tf.nn.sigmoid(z1)
z2 = tf.matmul(fc1, w2) + b2

loss = tf.reduce_mean(tf.nn.l2_loss(z2 - y_))
op = tf.train.AdamOptimizer(1).minimize(loss)  # increase learning rate

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("initial b2 weights: ", sess.run(b2))
    
    # use learned weights for this new model
    w1.load(w1_new, sess)
    b1.load(b1_new, sess)
    w2.load(w2_new, sess)
    b2.load(b2_new, sess)

    print("desired b2 weights: ", b2_new)
    print("new b2 weights: ", sess.run(b2))  # should match desired b2 weights

In [ ]:
# generate points - but this time only generate ~5% of datapoints (sparse data)
n_points_per_samples = int(n_points_per_samples / 20)

X_test_subset, y_test_subset, X_test, y_test = generate_points(n_points_per_samples)
plt.plot(X_test_subset, y_test_subset, c='g', marker='o')

print("number of points: ", n_points_per_samples)

In [ ]:
n_epochs = 2000  # same number of training steps, but on less data

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # load previous weights
    w1.load(w1_new, sess)
    b1.load(b1_new, sess)
    w2.load(w2_new, sess)
    b2.load(b2_new, sess)
    
    # train on a subset of test set - only a few data points
    for i in range(n_epochs):
        l_, _ = sess.run([loss, op], feed_dict={X_:X_test_subset.reshape(n_points_per_samples,1), 
                                                y_:y_test_subset.reshape(n_points_per_samples,1)})
        
    y_pred = sess.run(z2, feed_dict={X_:X_test.reshape(X_test.shape[0],1)})
    print("loss: ", l_)
    
    y_pred_subset = sess.run(z2, feed_dict={X_:X_test_subset.reshape(X_test_subset.shape[0],1)})

In [ ]:
plt.plot(X_test, y_test, c='g')  # actual
plt.plot(X_test, y_pred, c='b')  # predictions
plt.plot(X_test_subset, y_pred_subset, c='b', marker='x')